### Whisp a feature collection

Setup
- NB use a virtual environment to avoid altering your python environment (https://docs.python.org/3/tutorial/venv.html)

In [2]:
# Earth Engine and Common Libraries
import ee
from pathlib import Path

# Authenticate and initialize Earth Engine
try:
    ee.Initialize()  # Try to use existing credentials first
except Exception:
    ee.Authenticate()
    ee.Initialize()

In [8]:
# Install openforis-whisp (if not already installed)
!pip install --pre openforis-whisp

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import openforis_whisp as whisp

2025-04-09 10:25:48,753 - INFO - NumExpr defaulting to 2 threads.


Get a feature collection

In [4]:
#GEOJSON_EXAMPLE_FILEPATH = whisp.get_example_data_path("geojson_example.geojson")
GEOJSON_EXAMPLE_FILEPATH = whisp.get_example_data_path("foret_agrimey.geojson")
GEOJSON_EXAMPLE_FILEPATH = whisp.get_example_data_path("contour_AAC_ASI-Gabon.geojson")



print (GEOJSON_EXAMPLE_FILEPATH)

../tests/fixtures/contour_AAC_ASI-Gabon.geojson


Whisp it

In [5]:
df_formatted_stats = whisp.whisp_formatted_stats_geojson_to_df(GEOJSON_EXAMPLE_FILEPATH)

Reading GeoJSON file from: /home/sepal-user/whisp/tests/fixtures/contour_AAC_ASI-Gabon.geojson
['Area', 'European_Primary_Forest', 'GLC_FCS30D_TC_2022', 'GLC_FCS30D_crop_2022', 'IFL_2020', 'IIASA_planted_plantation', 'Cocoa_bnetd', 'Oil_palm_Descals', 'ESA_fire_before_2020', 'ESA_fire_2001', 'ESA_fire_2002', 'ESA_fire_2003', 'ESA_fire_2004', 'ESA_fire_2005', 'ESA_fire_2006', 'ESA_fire_2007', 'ESA_fire_2008', 'ESA_fire_2009', 'ESA_fire_2010', 'ESA_fire_2011', 'ESA_fire_2012', 'ESA_fire_2013', 'ESA_fire_2014', 'ESA_fire_2015', 'ESA_fire_2016', 'ESA_fire_2017', 'ESA_fire_2018', 'ESA_fire_2019', 'ESA_fire_2020', 'ESA_TC_2020', 'ESRI_2023_TC', 'ESRI_2023_crop', 'Cocoa_ETH', 'Cocoa_2023_FDaP', 'Cocoa_FDaP', 'Forest_FDaP', 'Oil_palm_2023_FDaP', 'Oil_palm_FDaP', 'Rubber_2023_FDaP', 'Rubber_FDaP', 'GFT_naturally_regenerating', 'GFT_planted_plantation', 'GFT_primary', 'GFC_TC_2020', 'GFC_loss_after_2020', 'GFC_loss_before_2020', 'GFC_loss_year_2001', 'GFC_loss_year_2002', 'GFC_loss_year_2003', '

Display table

In [6]:
df_formatted_stats

,plotId,external_id,Area,Geometry_type,Country,ProducerCountry,Admin_Level_1,Centroid_lon,Centroid_lat,Unit,...,TMF_regrowth_2023,ESRI_2023_TC,GLC_FCS30D_TC_2022,Oil_palm_2023_FDaP,Rubber_2023_FDaP,Cocoa_2023_FDaP,ESRI_2023_crop,GLC_FCS30D_crop_2022,GFW_logging,geo
0,1,NaN,9043.37207,Polygon,GAB,GA,Ngounié,10.89905,-1.024023,ha,...,0.715,9039.518555,8500.275391,0.0,0.0,0.0,0.0,0.894,0.0,"{'type': 'Polygon', 'coordinates': [[[10.86284..."


In [7]:
# Define the output folder
out_directory = Path.home() / 'module_results/whisp'

# Define the output file path for CSV
csv_output_file = out_directory / 'whisp_output_table_stats.csv'

# Save the CSV file
df_formatted_stats.to_csv(path_or_buf=csv_output_file, index=False)
print(f"Table with risk columns saved to: {csv_output_file}")

Table with risk columns saved to: /home/sepal-user/module_results/whisp/whisp_output_table_stats.csv


Calculate risk category

In [8]:
# add risk columns to end of dataframe
df_w_risk = whisp.whisp_risk(df=df_formatted_stats)

Display table with risk columns

In [9]:
df_w_risk

,plotId,external_id,Area,Geometry_type,Country,ProducerCountry,Admin_Level_1,Centroid_lon,Centroid_lat,Unit,...,Ind_05_primary_2020,Ind_06_nat_reg_forest_2020,Ind_07_planted_plantations_2020,Ind_08_planted_plantations_post_2020,Ind_09_treecover_post_2020,Ind_10_agri_post_2020,Ind_11_logging_concession,risk_pcrop,risk_acrop,risk_timber
0,1,NaN,9043.37207,Polygon,GAB,GA,Ngounié,10.89905,-1.024023,ha,...,yes,yes,no,no,yes,no,no,more_info_needed,more_info_needed,low


Export table to CSV

In [16]:
# Define the output folder
out_directory = Path.home() / 'module_results/whisp/'
#out_directory = Path.home() / 'Downloads'

# Define the output file path for CSV
csv_output_file = out_directory / 'whisp_foret_contour_AAC_ASI-Gabon_output_table_w_risk.csv'

# Save the CSV file
df_w_risk.to_csv(path_or_buf=csv_output_file, index=False)
print(f"Table with risk columns saved to: {csv_output_file}")

Table with risk columns saved to: /home/sepal-user/module_results/whisp/whisp_foret_contour_AAC_ASI-Gabon_output_table_w_risk.csv


Export to GeoJSON (optional)

In [ ]:
# Define the output file path for GeoJSON
geojson_output_file = out_directory / 'whisp_output_table.geojson'

# Save the GeoJSON file
whisp.convert_df_to_geojson(df_w_risk, geojson_output_file)  # builds a geojson file containing Whisp columns. Uses the geometry column "geo" to create the spatial features.
print(f"GeoJSON file saved to: {geojson_output_file}")